# Breast Cancer Recurrance Prediction Fuzzy System Implementation
First we install the necessary libraries

In [1]:
%pip install pandas
%pip install numpy
%pip install -U scikit-fuzzy


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Then we import the following libraries:
* pandas: To manipulate dataframe
* numpy: To perform array calculations
* scikit-fuzzy: To implement the fuzzy system
* control: To determine the fuzzy variables

In [2]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

Then we load the dataset from the uciml repository. We will be using the Breast Cancer Data UCI Repository

In [4]:
from ucimlrepo import fetch_ucirepo 
breast_cancer = fetch_ucirepo(id=14)
X = breast_cancer.data.features 
y = breast_cancer.data.targets 
data = X
data['Class'] = y  # Adding the target variable 'Class'
data

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,30-39,premeno,30-34,0-2,no,3,left,left_low,no,no-recurrence-events
1,40-49,premeno,20-24,0-2,no,2,right,right_up,no,no-recurrence-events
2,40-49,premeno,20-24,0-2,no,2,left,left_low,no,no-recurrence-events
3,60-69,ge40,15-19,0-2,no,2,right,left_up,no,no-recurrence-events
4,40-49,premeno,0-4,0-2,no,2,right,right_low,no,no-recurrence-events
...,...,...,...,...,...,...,...,...,...,...
281,30-39,premeno,30-34,0-2,no,2,left,left_up,no,recurrence-events
282,30-39,premeno,20-24,0-2,no,3,left,left_up,yes,recurrence-events
283,60-69,ge40,20-24,0-2,no,1,right,left_up,no,recurrence-events
284,40-49,ge40,30-34,5-Mar,no,3,left,left_low,no,recurrence-events


Then we will clean the data:
* Select only the age, tumor-size, inv-nodes, deg-malig and Class columns
* For tumor-size and inv-nodes, some data is imported as dates. Convert these dates (EG: 5-Mar) to range (EG: 3-5).
* Clean the age, tumor-size and inv-nodes column by replacing range (EG: 3-5) with midpoint (EG: 4)
* Clean Class column by replacing no-recurrence-events and recurrence-events with 0 and 1 respectively
* Convert datatype of age, tumor-size,inv-nodes and Class to float
* Save clean data as CSV file named "clean-breast-cancer.csv"

In [5]:
#Select only the age, tumor-size, inv-nodes, deg-malig and Class columns
data_clean = data[["age","tumor-size","inv-nodes", "deg-malig", "Class"]]

from datetime import datetime
for index, row in data_clean.iterrows():

    #Clean age Column
    age_list = row["age"].split("-")    #Extract range values
    age_average = (float(age_list[0]) + float(age_list[1])) / 2 #Calculate midpoint
    data_clean.at[index, "age"] = age_average   #Replacing range with age midpoint

    #Clean tumor-size column
    tumor_size_list = row["tumor-size"].split("-")  #Extract range values
    try:    #Convert these dates to range
        tumor_size_list[1] = datetime.strptime(tumor_size_list[1], '%b').month
    except:
        pass
    tumor_size_average = (float(tumor_size_list[0]) + float(tumor_size_list[1])) /2 #Calculate midpoint
    data_clean.at[index, "tumor-size"] = tumor_size_average #Replacing range with age midpoint

    #Clean inv-nodes column
    inv_nodes_list = row["inv-nodes"].split("-")  #Extract range values
    try:    #Convert these dates to range
        inv_nodes_list[1] = datetime.strptime(inv_nodes_list[1], '%b').month
    except:
        pass
    inv_nodes_average = (float(inv_nodes_list[0]) + float(inv_nodes_list[1])) /2    #Calculate midpoint
    data_clean.at[index, "inv-nodes"] = inv_nodes_average   #Replacing range with age midpoint
    
    #Clean Class column
    data_clean.at[index, "Class"] = 0 if row["Class"] == "no-recurrence-events" else 1  #replace no-recurrence-events and recurrence-events with 0 and 1 respectively

#Convert datatype of age, tumor-size,inv-nodes and Class to float
data_clean["age"] = data_clean["age"].astype(float)
data_clean["tumor-size"] = data_clean["tumor-size"].astype(float)
data_clean["inv-nodes"] = data_clean["inv-nodes"].astype(float)
data_clean["Class"] = data_clean["Class"].astype(float)

#Save clean data as CSV file named "clean-breast-cancer.csv"
data_clean.to_csv('clean-breast-cancer.csv', index=False, header=True)

#Display the clean data
data_clean

C:\Users\user\AppData\Local\Temp\ipykernel_188\2417224773.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean["age"] = data_clean["age"].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_188\2417224773.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean["tumor-size"] = data_clean["tumor-size"].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_188\2417224773.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,age,tumor-size,inv-nodes,deg-malig,Class
0,34.5,32.0,1.0,3,0.0
1,44.5,22.0,1.0,2,0.0
2,44.5,22.0,1.0,2,0.0
3,64.5,17.0,1.0,2,0.0
4,44.5,2.0,1.0,2,0.0
...,...,...,...,...,...
281,34.5,32.0,1.0,2,1.0
282,34.5,22.0,1.0,3,1.0
283,64.5,22.0,1.0,1,1.0
284,44.5,32.0,4.0,3,1.0


Then we define the fuzzy variables:
* age: 10 - 100
* tumor size = 0 - 60
* number of inv nodes = 0 - 40
* degree of malignance = 1-3
* recurrence = 0 - 100

In [6]:
# Define fuzzy variables
age = ctrl.Antecedent(np.arange(10, 101, 1), 'age')
tumor_size = ctrl.Antecedent(np.arange(0, 61, 1), 'tumor_size')
inv_nodes = ctrl.Antecedent(np.arange(0, 41, 1), 'inv_nodes')
deg_malig = ctrl.Antecedent(np.arange(1, 4, 1), 'deg_malig')
recurrence = ctrl.Consequent(np.arange(0, 101, 1), 'recurrence')

Then we define the membership functions for each variable

In [7]:
#Age membership functions
age['young'] = fuzz.trapmf(age.universe, [10, 10, 25, 50])
age['middle_aged'] = fuzz.trimf(age.universe, [25, 50, 70])
age['elderly'] = fuzz.trapmf(age.universe, [50, 70, 100, 100])

#Tumor size membership function
tumor_size['small'] = fuzz.trapmf(tumor_size.universe, [0, 0, 2, 10])
tumor_size['medium'] = fuzz.trimf(tumor_size.universe, [2, 10, 18])
tumor_size['large'] = fuzz.trapmf(tumor_size.universe, [10, 18, 60, 60])

#inv nodes membership function
inv_nodes['few'] = fuzz.trapmf(inv_nodes.universe, [0, 0, 2, 10])
inv_nodes['moderate'] = fuzz.trimf(inv_nodes.universe, [2, 10, 20])
inv_nodes['many'] = fuzz.trapmf(inv_nodes.universe, [10, 20, 40, 40])

#deg_malig membership function
deg_malig['low'] = fuzz.trimf(deg_malig.universe, [1, 1, 4])
deg_malig['high'] = fuzz.trimf(deg_malig.universe, [2, 4, 4])

# Define membership functions for the output (recurrence)
recurrence['low'] = fuzz.trimf(recurrence.universe, [0, 0, 50])
recurrence['medium'] = fuzz.trimf(recurrence.universe, [0, 50, 100])
recurrence['high'] = fuzz.trimf(recurrence.universe, [50, 100, 100])


Then we define the fuzzy rules. <br>
After that, we add the rules to the control system. <br>
Finally, we create a fuzzy system simulation based on those controls.

In [11]:
# Define fuzzy rules based on the membership functions and reasoning
rule =[]
rule.append(ctrl.Rule(age['young'], recurrence['high']))
rule.append(ctrl.Rule(age['middle_aged'], recurrence['medium']))
rule.append(ctrl.Rule(age['elderly'], recurrence['medium']))

rule.append(ctrl.Rule(inv_nodes['few'], recurrence['low']))
rule.append(ctrl.Rule(inv_nodes['moderate'], recurrence['medium']))
rule.append(ctrl.Rule(inv_nodes['many'], recurrence['high']))

rule.append(ctrl.Rule(deg_malig['low'], recurrence['low']))
rule.append(ctrl.Rule(deg_malig['high'], recurrence['high']))

rule.append(ctrl.Rule(tumor_size['small'], recurrence['low']))
rule.append(ctrl.Rule(tumor_size['medium'], recurrence['medium']))
rule.append(ctrl.Rule(tumor_size['large'], recurrence['high']))

rule.append(ctrl.Rule(inv_nodes['few'] & tumor_size['large'], recurrence['medium']))
rule.append(ctrl.Rule(deg_malig['high'] & tumor_size['large'], recurrence['high']))
rule.append(ctrl.Rule(age['middle_aged'] & inv_nodes['few'], recurrence['low']))
rule.append(ctrl.Rule(age['middle_aged'] & inv_nodes['moderate'], recurrence['low']))

#add the rules to the control system
recurrence_ctrl = ctrl.ControlSystem(rules=rule)

# Create simulation
recurrence_sim = ctrl.ControlSystemSimulation(recurrence_ctrl)

After that, we evaluate the fuzzy system by passing the clean data into the fuzzy system <br>
We will calculate metrics

In [9]:
count = 0 
correct = 0
for index, row in data_clean.iterrows():
    recurrence_sim.input['age'] = row["age"]
    recurrence_sim.input['tumor_size'] = row["tumor-size"]
    recurrence_sim.input['inv_nodes'] = row["inv-nodes"]
    recurrence_sim.input['deg_malig'] = row["deg-malig"]
    recurrence_sim.compute()
    crisp_output = 1 if recurrence_sim.output['recurrence'] > 50 else 0
    correct = correct+1 if crisp_output == row["Class"] else correct
    count += 1
    string = "Correct" if crisp_output == row["Class"] else "Wrong"
    if (string == "Wrong"):
        print(f"Data number {count}: age={row['age']}, tumor_size={row['tumor-size']}, inv_nodes={row['inv-nodes']}, deg-malig={row['deg-malig']}")
        print(f"Percentage {recurrence_sim.output['recurrence']}%, Output {crisp_output}, Answer {row['Class']}, {string}")

print(f"Number of correct diagnoses {correct}") 
print(f"Number of wrong diagnoses {count-correct}") 
print(f"Total diagnoses {count}") 
print(f"Accuracy {correct/count*100}%") 

Data number 127: age=34.5, tumor_size=32.0, inv_nodes=7.0, deg-malig=2.0
Percentage 51.89377126793902%, Output 1, Answer 0.0, Wrong
Data number 128: age=34.5, tumor_size=27.0, inv_nodes=7.0, deg-malig=2.0
Percentage 51.89377126793902%, Output 1, Answer 0.0, Wrong
Data number 130: age=44.5, tumor_size=37.0, inv_nodes=10.0, deg-malig=2.0
Percentage 50.75976871301438%, Output 1, Answer 0.0, Wrong
Data number 131: age=44.5, tumor_size=37.0, inv_nodes=10.0, deg-malig=2.0
Percentage 50.75976871301438%, Output 1, Answer 0.0, Wrong
Data number 132: age=44.5, tumor_size=42.0, inv_nodes=4.0, deg-malig=3.0
Percentage 51.00840755212913%, Output 1, Answer 0.0, Wrong
Data number 133: age=44.5, tumor_size=32.0, inv_nodes=7.0, deg-malig=2.0
Percentage 51.76866561254192%, Output 1, Answer 0.0, Wrong
Data number 136: age=34.5, tumor_size=22.0, inv_nodes=4.0, deg-malig=2.0
Percentage 51.0185185185185%, Output 1, Answer 0.0, Wrong
Data number 137: age=34.5, tumor_size=42.0, inv_nodes=4.0, deg-malig=3.0
Pe

In [10]:
import pandas as pd

# Create an empty DataFrame to store wrong diagnoses
wrong_diagnoses = pd.DataFrame(columns=['Age', 'Tumor_Size', 'Inv_Nodes', 'Deg_Malig', 'Percentage', 'Output', 'Answer'])

count = 0
correct = 0
for index, row in data_clean.iterrows():
    recurrence_sim.input['age'] = row["age"]
    recurrence_sim.input['tumor_size'] = row["tumor-size"]
    recurrence_sim.input['inv_nodes'] = row["inv-nodes"]
    recurrence_sim.input['deg_malig'] = row["deg-malig"]
    recurrence_sim.compute()
    crisp_output = 1 if recurrence_sim.output['recurrence'] > 50 else 0
    correct = correct + 1 if crisp_output == row["Class"] else correct
    count += 1
    string = "Correct" if crisp_output == row["Class"] else "Wrong"
    if string == "Wrong":
        wrong_diagnoses = wrong_diagnoses.append({
            'Age': row['age'],
            'Tumor_Size': row['tumor-size'],
            'Inv_Nodes': row['inv-nodes'],
            'Deg_Malig': row['deg-malig'],
            'Percentage': recurrence_sim.output['recurrence'],
            'Output': crisp_output,
            'Answer': row['Class']
        }, ignore_index=True)

        print(f"Data number {count}: age={row['age']}, tumor_size={row['tumor-size']}, inv_nodes={row['inv-nodes']}, deg-malig={row['deg-malig']}")
        print(f"Percentage {recurrence_sim.output['recurrence']}%, Output {crisp_output}, Answer {row['Class']}, {string}")

print(f"Number of correct diagnoses {correct}") 
print(f"Number of wrong diagnoses {count - correct}") 
print(f"Total diagnoses {count}") 
print(f"Accuracy {correct / count * 100}%")

# Now 'wrong_diagnoses' DataFrame contains the wrong diagnosis entries


Data number 127: age=34.5, tumor_size=32.0, inv_nodes=7.0, deg-malig=2.0
Percentage 51.89377126793902%, Output 1, Answer 0.0, Wrong
Data number 128: age=34.5, tumor_size=27.0, inv_nodes=7.0, deg-malig=2.0
Percentage 51.89377126793902%, Output 1, Answer 0.0, Wrong
Data number 130: age=44.5, tumor_size=37.0, inv_nodes=10.0, deg-malig=2.0
Percentage 50.75976871301438%, Output 1, Answer 0.0, Wrong
Data number 131: age=44.5, tumor_size=37.0, inv_nodes=10.0, deg-malig=2.0
Percentage 50.75976871301438%, Output 1, Answer 0.0, Wrong
Data number 132: age=44.5, tumor_size=42.0, inv_nodes=4.0, deg-malig=3.0
Percentage 51.00840755212913%, Output 1, Answer 0.0, Wrong
Data number 133: age=44.5, tumor_size=32.0, inv_nodes=7.0, deg-malig=2.0
Percentage 51.76866561254192%, Output 1, Answer 0.0, Wrong
Data number 136: age=34.5, tumor_size=22.0, inv_nodes=4.0, deg-malig=2.0
Percentage 51.0185185185185%, Output 1, Answer 0.0, Wrong
Data number 137: age=34.5, tumor_size=42.0, inv_nodes=4.0, deg-malig=3.0
Pe

C:\Users\user\AppData\Local\Temp\ipykernel_6804\2613582134.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wrong_diagnoses = wrong_diagnoses.append({
C:\Users\user\AppData\Local\Temp\ipykernel_6804\2613582134.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wrong_diagnoses = wrong_diagnoses.append({
C:\Users\user\AppData\Local\Temp\ipykernel_6804\2613582134.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wrong_diagnoses = wrong_diagnoses.append({
C:\Users\user\AppData\Local\Temp\ipykernel_6804\2613582134.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wrong_diagnoses = wrong_diagnoses.append({
C:\Users\user\AppData\Lo

In [11]:

wrong_diagnoses.to_csv('wrong-diagnosis.csv', index=False, header=True)


In [12]:
wrong_diagnoses

,Age,Tumor_Size,Inv_Nodes,Deg_Malig,Percentage,Output,Answer
0,34.5,32.0,7.0,2.0,51.893771,1.0,0.0
1,34.5,27.0,7.0,2.0,51.893771,1.0,0.0
2,44.5,37.0,10.0,2.0,50.759769,1.0,0.0
3,44.5,37.0,10.0,2.0,50.759769,1.0,0.0
4,44.5,42.0,4.0,3.0,51.008408,1.0,0.0
...,...,...,...,...,...,...,...
73,44.5,17.0,1.0,3.0,49.746473,0.0,1.0
74,64.5,32.0,1.0,2.0,50.000000,0.0,1.0
75,34.5,32.0,1.0,2.0,50.000000,0.0,1.0
76,34.5,22.0,1.0,3.0,50.000000,0.0,1.0
